In [2]:
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, f1_score


In [3]:
# Load training data
data_path = '/workspaces/Final-Year-Project/Cleaned Data/TrainTestData.csv'
df = pd.read_csv(data_path)

# Display the original shape of the dataset
print(f"🔍 SVM - Original dataset shape: {df.shape}")

# Drop missing values
df = df.dropna()

# Display the shape after dropping NaNs
print(f"🧹 SVM - After dropping NaNs: {df.shape}")

# Features and target variable
X = df.drop('Diabetes Status', axis=1)
y = df['Diabetes Status']


🔍 SVM - Original dataset shape: (3268, 14)
🧹 SVM - After dropping NaNs: (3268, 14)


In [4]:
# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Display the shape after scaling
print(f"📊 SVM - Final dataset shape after scaling: {X_scaled.shape}")


📊 SVM - Final dataset shape after scaling: (3268, 13)


In [5]:
# SVM Classifier setup
svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# KFold cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get the accuracy scores
svm_kfold_scores = cross_val_score(svm, X_scaled, y, cv=kf, scoring='accuracy')

# Print the K-Fold Mean Accuracy
print(f"✅ SVM - K-Fold Mean Accuracy: {svm_kfold_scores.mean():.4f}")


✅ SVM - K-Fold Mean Accuracy: 0.7423


In [6]:
# Fit SVM model on the training data
svm.fit(X_scaled, y)

# Predictions on training data
y_pred = svm.predict(X_scaled)

# Classification report on the training data
print("📌 SVM Classification Report on Training Data:")
print(classification_report(y, y_pred))

# Confusion matrix on the training data
print("🟦 SVM - Confusion Matrix on Training Data:")
print(confusion_matrix(y, y_pred))


📌 SVM Classification Report on Training Data:
              precision    recall  f1-score   support

           0       0.83      0.68      0.75      1638
           1       0.73      0.86      0.79      1630

    accuracy                           0.77      3268
   macro avg       0.78      0.77      0.77      3268
weighted avg       0.78      0.77      0.77      3268

🟦 SVM - Confusion Matrix on Training Data:
[[1111  527]
 [ 226 1404]]


In [7]:
# Hyperparameter grid for GridSearchCV
param_grid = {
    'C': [0.0001],  # Regularization strength (try multiple values if needed)
}

# GridSearchCV setup with accuracy as the scoring parameter
grid_search = GridSearchCV(SVC(), param_grid, cv=kf, n_jobs=-1, verbose=2, scoring='accuracy')

# Fit GridSearchCV on the training data
grid_search.fit(X_scaled, y)

# Print the best parameters and the best score from GridSearchCV
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...........................................C=0.0001; total time=   0.5s
[CV] END ...........................................C=0.0001; total time=   0.5s
[CV] END ...........................................C=0.0001; total time=   0.5s
[CV] END ...........................................C=0.0001; total time=   0.6s
[CV] END ...........................................C=0.0001; total time=   0.4s
Best Parameters: {'C': 0.0001}
Best Cross-Validation Accuracy: 0.4902


In [8]:
# Load validation data
valid_path = "/workspaces/Final-Year-Project/Cleaned Data/ValidationData.csv"
df_valid = pd.read_csv(valid_path)

# Drop any rows with missing values in the validation set
df_valid = df_valid.dropna()

# Prepare features and target for validation data
X_valid = df_valid.drop('Diabetes Status', axis=1)
y_valid = df_valid['Diabetes Status']

# Scale the validation data using the previously fitted scaler
X_valid_scaled = scaler.transform(X_valid)

# Display the shape of the validation dataset
print(f"🧪 SVM - Validation dataset shape: {X_valid.shape}")


🧪 SVM - Validation dataset shape: (364, 13)


In [9]:
# Predictions on validation data
y_pred_valid = svm.predict(X_valid_scaled)


In [10]:
# Print classification report for validation data
print(f"📌 SVM - Validation Classification Report:")
print(classification_report(y_valid, y_pred_valid))

# Print confusion matrix for validation data
print("🟦 SVM - Validation Confusion Matrix:")
print(confusion_matrix(y_valid, y_pred_valid))


📌 SVM - Validation Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.66      0.73       178
           1       0.72      0.84      0.78       186

    accuracy                           0.76       364
   macro avg       0.76      0.75      0.75       364
weighted avg       0.76      0.76      0.75       364

🟦 SVM - Validation Confusion Matrix:
[[118  60]
 [ 29 157]]


In [11]:
import pandas as pd
from sklearn.metrics import classification_report

# Assuming you already have y_valid (true labels) and y_pred_valid (predicted labels)
# Example classification report for AdaBoost (AB)
report = classification_report(y_valid, y_pred_valid, output_dict=True)

# Extract weighted scores
weighted_precision = report['weighted avg']['precision']
weighted_recall = report['weighted avg']['recall']
weighted_f1 = report['weighted avg']['f1-score']
accuracy = report['accuracy']

# Create a dictionary with the scores
ab_results = {
    'Model': 'Support Vecotr Machine(SVM)',
    'Precision': weighted_precision,
    'Recall': weighted_recall,
    'F1-Score': weighted_f1,
    'Accuracy': accuracy
}

# Convert the dictionary to a DataFrame
ab_df = pd.DataFrame([ab_results])

# Create a DataFrame to store results (if not already created)
results_df = pd.read_csv("/workspaces/Final-Year-Project/Results/model_results.csv")

# Concatenate the new row to the DataFrame
results_df = pd.concat([results_df, ab_df], ignore_index=True)

# Print the DataFrame to confirm it's added
print(results_df)

# Save the DataFrame to a CSV file
results_df.to_csv("/workspaces/Final-Year-Project/Results/model_results.csv", index=False)


                               Model  Precision    Recall  F1-Score  Accuracy
0                      AdaBoost (AB)   0.767212  0.752747  0.748389  0.752747
1           Logistic Regression (LR)   0.753372  0.750000  0.748645  0.750000
2                    Naive Bayes(LR)   0.763240  0.755495  0.752966  0.755495
3  K Nearest Nearest Neighbours(KNN)   0.723548  0.722527  0.721802  0.722527
4                  Decision Tree(DT)   0.748498  0.741758  0.739233  0.741758
5                  Random Forest(RF)   0.746618  0.744505  0.743511  0.744505
6        Support Vecotr Machine(SVM)   0.762241  0.755495  0.753238  0.755495


In [12]:
# Recalculate Macro average F1-score on validation data
validation_f1_macro = f1_score(y_valid, y_pred_valid, average='macro')
print(f"🏆 Best Model Validation F1-Score: {validation_f1_macro}")


🏆 Best Model Validation F1-Score: 0.7526550868486352
